<a href="https://colab.research.google.com/github/lisethrubio/Project4-Machine_Learning_Model/blob/Steven_branch2/Health_Prediction_Model_Class_x_8dim_inc_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

In [2]:
# Import our input dataset
body_df = pd.read_csv('/content/bodyPerformance.csv')
body_df.head()

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B


In [3]:
# Encode (convert to dummy variables) the "gender" column
gender_encoded = pd.get_dummies(body_df['gender'], prefix="gender", dtype=int)

In [4]:
gender_encoded.head()

,gender_F,gender_M
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [5]:
# Concatenate the scaled data DataFrame with the "IMF Country Code" encoded dummies
new_body_df = pd.concat([body_df, gender_encoded], axis=1)


In [6]:
# Encode the target variable 'class'
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(new_body_df["class"].values)

# Remove 'class' and 'gender' columns from the features data
X = new_body_df.drop(columns=["class", "gender", "gripForce", "sit and bend forward_cm", "sit-ups counts", "broad jump_cm"]).values

In [7]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [8]:
# Preprocess numerical data for the neural network
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=4)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=4)

In [11]:
# Define the deep learning model
nn_model = tf.keras.models.Sequential()

# Update input_dim to match the number of input features (10)
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=8))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer for 4 classes with softmax activation
nn_model.add(tf.keras.layers.Dense(units=4, activation="softmax"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.3448 - loss: 1.3472
Epoch 2/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4087 - loss: 1.2452
Epoch 3/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4260 - loss: 1.2307
Epoch 4/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.4360 - loss: 1.2215
Epoch 5/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4409 - loss: 1.2121
Epoch 6/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4390 - loss: 1.2128
Epoch 7/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4368 - loss: 1.2049
Epoch 8/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4512 - loss: 1.1880
Epoch 9/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4476 - loss: 1.1971
Epoch 10/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4459 - loss: 1.2056
Epoch 11/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4422 - loss: 1.1944
Epoch 12/100
314/314 ━━━━━━━━━━━━━━━━━━━━

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

105/105 - 0s - 3ms/step - accuracy: 0.4539 - loss: 1.1863
Loss: 1.1862621307373047, Accuracy: 0.4538668394088745
